In [18]:
import torchvision.transforms as T
from torch.utils.data import Subset, Dataset, DataLoader, random_split
from torch import nn
import torch.nn.functional
from torchvision.datasets import ImageFolder
from torchvision.io import read_image, ImageReadMode
from torchvision import models
import torch
import pickle as pkl
import os
from PIL import Image


<h1>Trying a predefined model (Resnet)</h1>

In [2]:

dataset_path = "../informatie/apple_disease_classification/images/Train/Dataset/"

In [3]:
class DatasetAppels(Dataset):
    def __init__(self, img_folder_path, transform):

        image_folder = ImageFolder(img_folder_path, transform=transform)
        self.images = [image[0] for image in image_folder]
        self.labels = image_folder.targets
        self.class_dict = image_folder.class_to_idx

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        return [self.images[idx], self.labels[idx]]
        

In [4]:
preprocess = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [5]:
dataset = DatasetAppels(dataset_path, preprocess)

In [6]:
generator1 = torch.Generator().manual_seed(13)

# create a train test split with 60% train, 20% test, 20% val. For later use
train_dataset, test_dataset, val_dataset, _ = random_split(dataset, [0.2, 0.2, 0.2, 0.4], generator=generator1)
print(len(train_dataset), len(test_dataset), len(val_dataset))

# Create train, test and val dataloaders for later use.
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)

test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

val_loader = DataLoader(val_dataset, batch_size=10, shuffle=False)

292 292 291


In [7]:
# Resnet variants:
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet152', pretrained=False)

In [15]:
class Resnet50():
    def __init__(self):      
        self.model = models.resnet50(weights='IMAGENET1K_V1')
        num_ftrs = self.model.fc.in_features

        # set number of output classes

        self.model.fc = nn.Linear(num_ftrs, 4)
        self.model = self.model.to("cuda")
        self.loss_func = nn.CrossEntropyLoss()

    # Function for cuda
    def cuda_available(self):

        if torch.cuda.is_available():
            return torch.device("cuda")
        else:
            return torch.device("cpu")
        
    # Create a function to calculate the loss
    def loss_calc(self, batch):
        image = batch[0].to(self.cuda_available())
        labels = batch[1].to(self.cuda_available())
        pred = self.model(image)        
        loss = self.loss_func(pred, labels)
        return loss
    
    # Create function to evaluate the accuracy of my model
    def evaluate_accuracy(self, test_loader):
        cor_pred = 0
        bad_pred = 0
        for batch in test_loader:
            image, labels = batch
            image = image.to(self.cuda_available())
            labels = labels.to(self.cuda_available())
            pred = self.model(image)
    
            _, y_pred = torch.max(pred,1)

            for image, labels in zip(y_pred, labels):
                if image == labels:
                    cor_pred += 1
                else:
                    bad_pred += 1

        acc = cor_pred/(cor_pred + bad_pred) * 100
        return acc

    def fit(self, train_loader, val_loader, test_loader, epochs, lr, opt_function=torch.optim.Adam):
        
        optimizer = opt_function(self.model.parameters(),lr )
        # Set to cuda (gpu)
        self.model.to(self.cuda_available())
        # Create empty list to save the validation results
        history = []
        # Training loop
        for epoch in range(epochs):
            print("epoch:",epoch+1)
            self.model.train()
            
            for batch in train_loader:
                optimizer.zero_grad()
                # I use the function loss_calc that I created below so that I can use this at the validation.
                loss = self.loss_calc(batch)
                loss.backward()
                optimizer.step()

            # Validate
            with torch.no_grad():    
                self.model.eval()
                val_loss = []

                for batch in val_loader:
                    loss = self.loss_calc(batch)
                    val_loss.append(loss)  
            
            # Append the sum(val_loss)
            history.append(sum(val_loss))
            print(sum(val_loss))        
        
        # I use a self made function return a percentage of the accuracy
        acc = round(self.evaluate_accuracy(test_loader))
        print("Accuracy:",acc)
        return history, acc
    
    def predict_image(self, image):
        image = image.to(self.cuda_available())
        pred = self.model(image)

        _, y_pred = torch.max(pred,1)                
        result = y_pred

        return result

In [16]:
model = Resnet50()
model.fit(train_loader, val_loader, test_loader, lr = 0.00003, epochs=10)

epoch: 1
tensor(17.3222, device='cuda:0')
epoch: 2
tensor(9.3138, device='cuda:0')
epoch: 3
tensor(8.1125, device='cuda:0')
epoch: 4
tensor(7.3294, device='cuda:0')
epoch: 5
tensor(6.9509, device='cuda:0')
epoch: 6
tensor(8.0044, device='cuda:0')
epoch: 7
tensor(7.3569, device='cuda:0')
epoch: 8
tensor(6.6958, device='cuda:0')
epoch: 9
tensor(6.3838, device='cuda:0')
epoch: 10
tensor(8.3294, device='cuda:0')
Accuracy: 94


([tensor(17.3222, device='cuda:0'),
  tensor(9.3138, device='cuda:0'),
  tensor(8.1125, device='cuda:0'),
  tensor(7.3294, device='cuda:0'),
  tensor(6.9509, device='cuda:0'),
  tensor(8.0044, device='cuda:0'),
  tensor(7.3569, device='cuda:0'),
  tensor(6.6958, device='cuda:0'),
  tensor(6.3838, device='cuda:0'),
  tensor(8.3294, device='cuda:0')],
 94)

In [23]:
# Lets try with normal images
normal = [32, 33, 34]
print(dataset.class_dict)
for i in normal:
    test_path = f"../informatie/apple_disease_classification/images/Test/Normal_Apple/{i}.jpg"
    test_img = Image.open(test_path)
    test_img = preprocess(test_img)

    # Since the model expects a batch I have to use the unsqueeze() function
    test_img = test_img.unsqueeze(0)
    result = model.predict_image(test_img)
    print(result)


{'blotch_apples': 0, 'normal_apples': 1, 'rot_apples': 2, 'scab_apples': 3}
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
tensor([1], device='cuda:0')
